# EE447: Full State Feedback

Eric Klavins

Copyright &copy; University of Washington, 2019

# Code

In [1]:
import numpy as np
import scipy.integrate as spi
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sympy import *
from matplotlib import animation

# Uncomment on Google colab
# !pip install JSAnimation
# !pip install control

from JSAnimation.IPython_display import display_animation
from control import * 

%matplotlib inline

# Comment out in Google colab 
init_printing(use_latex='mathjax')

# Uncomment in below Google colab to render sympy equations nicely
# def custom_latex_printer(exp,**options):
#     from google.colab.output._publish import javascript
#     url = "https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.3/latest.js?config=default"
#     javascript(url=url)
#     return printing.latex(exp,**options)

def MatrixFromColumns(clist):
    return Matrix(clist).reshape(
        clist[0].shape[0], 
        len(clist)).transpose()

# Review

Last time we took a system

$$
\dot\x = A \x + B u
$$

and defined $u = -K\x + r$ to get

$$
\dot\x = (A-BK)\x + Br
$$

and we showed examples where we could define $K$ so that the poles of $A-BK$ are *whatever you want*. This is called **pole placement**.

We also saw an example 

\begin{align}
\dot \x & = \begin{pmatrix}
  -1 & 0 \\
  0 & -2
\end{pmatrix} \x + \begin{pmatrix}
0 \\
1
\end{pmatrix} u \\
y & = ( 0 \; 1 ) \; \x
\end{align}

where the approach did not work. This system is called **not completely controllable**.

# Definition of Controllability

*Def:* A system is **controllable** is for any $\x(0)$ and $\x^*$ there exists a $u(t)$ that will drive the system from $\x(0)$ to $\x^*$ in finite time. 

We will not prove the following results, which require more linear algebra than we have gone over in this course. If you are interested, take EE 547 or read a book like [Linear Systems](https://www.amazon.com/Linear-System-Electrical-Computer-Engineering/dp/0195117778) by Chi-Tsong Chen.

> **Theorem**: If a linear system is controllable, then $K$ can be defined to place the poles of $A-BK$ arbitrarily. 

What is needed is a *test* of controllability. The following result is one of the fundamental results in linear systems theory:

> **Theorem**: A system $\dot\x = A\x + Bu$ is controllable if and only if the matrix
$$
M = [ B \; AB \; A^2 B \; \dots \; A^{n-1}B ]
$$
has rank $n$, where $n$ is the dimension of $\x$. 

Reminder: The **rank** of a matrix is the largest number of independent columns. You can find the rank by finding the largest non-singular submatrix (which means determinant $\neq$ zero). If the matrix itself has a non-zero-determinant, then it has **full rank**.

Example
---

The example above with 

$$
\dot \x  = \begin{pmatrix}
  -1 & 0 \\
  0 & -2
\end{pmatrix} \x + \begin{pmatrix}
0 \\
1
\end{pmatrix} u 
$$

has 

$$
M = \left [
\begin{pmatrix}
0 \\ 
1
\end{pmatrix} \;\;\;
\begin{pmatrix}
  -1 & 0 \\
  0 & -2
\end{pmatrix}\begin{pmatrix}
0 \\
1
\end{pmatrix} 
\right ] = \begin{pmatrix}
0 & 0 \\
1 & -2 
\end{pmatrix}
$$

which has rank $1$, not $2$. Therefore, the system is not controllable.

An example where the system is controllable, is shown below using `sympy`.

In [106]:
#
# Example: Determine the controllability of A, B
#
A = Matrix([
    [1,2,1],
    [0,-1,0],
    [0,0,1]
])
B = Matrix([
    [0],
    [1],
    [1]
])
M = MatrixFromColumns([B,A*B,A*A*B])
M, M.rank()

⎛⎡0  3   2⎤, 3⎞
⎜⎢        ⎥   ⎟
⎜⎢1  -1  1⎥   ⎟
⎜⎢        ⎥   ⎟
⎝⎣1  1   1⎦   ⎠

In [21]:
# 
# A really random example
#

A = Matrix(np.random.randint(3,size=(4,4)))
B = Matrix(np.random.randint(3,size=(4,1)))
M = MatrixFromColumns([B,A*B,A**2*B, A**3*B])
M, M.rank()

⎛⎡0.0  2.0  6.0  20.0⎤, 2⎞
⎜⎢                   ⎥   ⎟
⎜⎢2.0   0   2.0  6.0 ⎥   ⎟
⎜⎢                   ⎥   ⎟
⎜⎢0.0   0    0    0  ⎥   ⎟
⎜⎢                   ⎥   ⎟
⎝⎣0.0  2.0  6.0  20.0⎦   ⎠

In [22]:
A,B

⎛⎡1.0  1.0  1.0  2.0⎤, ⎡0.0⎤⎞
⎜⎢                  ⎥  ⎢   ⎥⎟
⎜⎢1.0  0.0  1.0  0.0⎥  ⎢2.0⎥⎟
⎜⎢                  ⎥  ⎢   ⎥⎟
⎜⎢0.0  0.0  1.0  0.0⎥  ⎢0.0⎥⎟
⎜⎢                  ⎥  ⎢   ⎥⎟
⎝⎣2.0  1.0  0.0  1.0⎦  ⎣0.0⎦⎠

# Properties of $M$

The coolest thing about $M$ (besides that it tells you if your system is controllable), is that it helps you build $K$. Remember that $K$ is easy to build when $A\x + Bu$ is in phase canonical form. But what if it isn't? This is where $M$ comes in.

Say

$$
\dot\x = A \x + B u
$$

is *not* in phase canonical form. Say that

$$
\x = P {\bf z}
$$

where $P$ transforms ${\bf z}$ from phase canonical form into the coordinate system for $\x$. Then

$$
\dot {\bf z} = P^{-1}AP{\bf z} + P^{-1}B u.
$$

Now the controllability matrix in $\x$ coordinates is

$$
M_x = \left( B \; AB \; \dots \; A^{n-1}B \right)
$$

and in ${\bf z}$ coordinates is

\begin{align}
M_z & = \left ( P^{-1}B \;\; (P^{-1}AP)P^{-1}B \; \dots \; (P^{-1}AP)^{n-1}P^{-1}B \right ) \\
  & = P^{-1} \left( B \;\; AB \;\; \dots \; A^{n-1}B \right ) \\
  & = P^{-1} M_x
\end{align}

Thus, 

$$
P = M_x M_z^{-1}
$$

After transforming the system into phase variables, we design a controller

$$
u = -K_z {\bf z} + r
$$

to get 

\begin{align}
\dot {\bf z} & = (A_z - B_z K_z) {\bf z} + B_z r \\
 & = (P^{-1}AP - P^{-1}BK_z) {\bf z} + P^{-1}r
\end{align}

But since $\x = P{\bf z}$ we get

\begin{align}
\dot\x & = P \dot {\bf z} \\
       & = P (P^{-1}AP - P^{-1}BK_z) {\bf z} + P P^{-1}r\\
       & = (A-BK_zP^{-1})\x + Br \\
       & = (A-BK_x)\x + Br
\end{align}

where $K_x \triangleq K_z P^{-1}$.

An example using `sympy` is below.

In [118]:
#
# Example: Define K that A-BK has poles at -5, -6 where
#
A = Matrix([
    [-1,1],
    [0,-2]
])
B = Matrix([
    [1],
    [1]
])
Mx = MatrixFromColumns([B, A*B])
Mx, Mx.rank()

⎛⎡1  0 ⎤, 2⎞
⎜⎢     ⎥   ⎟
⎝⎣1  -2⎦   ⎠

In [119]:
#
# Characteristic polynomial 
#
var("s")
(s*eye(2)-A).det()

 2          
s  + 3⋅s + 2

In [127]:
#
# Z coordinates are just phase canonical form for 2 and 3.
#
Az = Matrix([
    [0,1],
    [-2,-3]
])
Bz = Matrix([
    [0],
    [1]
])
Mz = MatrixFromColumns([Bz, Az*Bz])
Az,Bz,Mz

⎛⎡0   1 ⎤, ⎡0⎤, ⎡0  1 ⎤⎞
⎜⎢      ⎥  ⎢ ⎥  ⎢     ⎥⎟
⎝⎣-2  -3⎦  ⎣1⎦  ⎣1  -3⎦⎠

In [133]:
#
# Design full state feedback controller in z coordinates.
# Note that (s+5)(s+6) = s^2 + 11s + 30
#
var("k0 k1")
sol = solve([3+k1-11,2+k0-30],[k0,k1])
Kz = Matrix([[k0,k1]]).subs(sol)
Kz

[28  8]

In [138]:
# Determine change of coordinates matrix
P = Mx*Mz.inv()
P

⎡3  1⎤
⎢    ⎥
⎣1  1⎦

In [139]:
# Get Kx from Kz
Kx = Kz*P.inv()
Kx

[10  -2]

In [141]:
# Check that qwe get the right characteristic polynomial
(s*eye(2)  - (A - B*Kx)).det()

 2            
s  + 11⋅s + 30

# Using the Control Toolbox

The above is so algorithmic, that it is encoded in the control systems toolbox:

In [25]:
Kx = place(
    [
        [-1,1],
        [0,-2]
    ],
    [
        [1],
        [1]
    ],
    [-5,-6]
)
Kx

array([[10., -2.]])